## 2.routing(google adk)

<img style="float: right;" src="../img/flex-logo.png" width="120"><br>

<div style="text-align: right"> <b>your name</b></div>
<div style="text-align: right"> Initial issue : 2025.10.02 </div>
<div style="text-align: right"> last update : 2025.10.02 </div>

개정 이력  
- `2025.10.02` : 노트북 

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import uuid
from typing import Dict, Any, Optional
from google.adk.agents import Agent
from google.adk.runners import InMemoryRunner
from google.adk.tools import FunctionTool
from google.genai import types
from google.adk.events import Event

1. 함수 정의: specialist agent 역할

In [3]:

def booking_handler(request: str) -> str:
    """
    항공권 및 호텔 예약 요청을 처리합니다.
    인자:
        request: 사용자의 예약 요청.
    반환:
        예약이 처리되었음을 확인하는 메시지.
    """
    print("-------------------------- 예약 핸들러 호출됨 ----------------------------")
    return f"'{request}'에 대한 예약 작업이 시뮬레이션되었습니다."

def info_handler(request: str) -> str:
    """
    일반 정보 요청을 처리합니다.
    인자:
        request: 사용자의 질문.
    반환:
        정보 요청이 처리되었음을 나타내는 메시지.
    """
    print("-------------------------- 정보 핸들러 호출됨 ----------------------------")
    return f"'{request}'에 대한 정보 요청. 결과: 정보 조회를 시뮬레이션했습니다."

def unclear_handler(request: str) -> str:
    """위임할 수 없는 요청을 처리합니다."""
    return f"코디네이터가 요청을 위임할 수 없습니다: '{request}'. 내용을 구체적으로 알려주세요."


툴 정의

In [4]:
booking_tool = FunctionTool(booking_handler)
info_tool = FunctionTool(info_handler)

2. Agent 정의

In [5]:
booking_agent = Agent(
    name="Booker",
    model="gemini-2.0-flash",
    description="예약 도구를 호출하여 항공권 및 호텔 예약 요청을 처리하는 특화 에이전트입니다.",
    tools=[booking_tool]
)

info_agent = Agent(
    name="Info",
    model="gemini-2.0-flash",
    description="정보 도구를 호출해 일반 정보를 제공하고 사용자 질문에 답하는 특화 에이전트입니다.",
    tools=[info_tool]
)

# Define the parent agent with explicit delegation instructions
coordinator = Agent(
    name="Coordinator",
    model="gemini-2.0-flash",
    instruction=(
        "당신은 메인 코디네이터입니다. 당신의 유일한 임무는 들어오는 사용자 요청을 분석하고 "
        "적절한 전문 에이전트에게 위임하는 것입니다. 사용자에게 직접 답변하려고 하지 마십시오.\n"
        "- 항공권 또는 호텔 예약과 관련된 모든 요청은 'Booker' 에이전트에 위임하세요.\n"
        "- 그 외 일반 정보 질문은 'Info' 에이전트에 위임하세요."
    ),
    description="사용자 요청을 올바른 전문 에이전트에게 라우팅하는 코디네이터입니다.",
    # The presence of sub_agents enables LLM-driven delegation (Auto-Flow) by default.
    sub_agents=[booking_agent, info_agent]
)

3. 실행 로직 정의

In [6]:
async def run_coordinator(runner: InMemoryRunner, request: str):
    """Runs the coordinator agent with a given request and delegates."""
    print(f"\n--- Running Coordinator with request: '{request}' ---")
    final_result = ""
    try:
        user_id = "user_123"
        session_id = str(uuid.uuid4())
        await runner.session_service.create_session(
            app_name=runner.app_name, user_id=user_id, session_id=session_id
        )

        for event in runner.run(
            user_id=user_id,
            session_id=session_id,
            new_message=types.Content(
                role='user',
                parts=[types.Part(text=request)]
            ),
        ):
            if event.is_final_response() and event.content:
                # Try to get text directly from event.content
                # to avoid iterating parts
                if hasattr(event.content, 'text') and event.content.text:
                     final_result = event.content.text
                elif event.content.parts:
                    # Fallback: Iterate through parts and extract text (might trigger warning)
                    text_parts = [part.text for part in event.content.parts if part.text]
                    final_result = "".join(text_parts)
                # Assuming the loop should break after the final response
                break

        print(f"Coordinator Final Response: {final_result}")
        return final_result
    except Exception as e:
        print(f"An error occurred while processing your request: {e}")
        return f"An error occurred while processing your request: {e}"

In [ ]:
async def main():
    """ADK 예제를 실행하는 메인 함수입니다."""
    print("--- Google ADK 라우팅 예제 (ADK 오토-플로우 스타일) ---")
    print("참고: Google ADK가 설치되어 있고 인증되어 있어야 합니다.")

    runner = InMemoryRunner(coordinator)

    # 사용 예
    result_a = await run_coordinator(runner, "파리에서 호텔을 예약해 주세요.")
    print(f"최종 출력 A: {result_a}")

    result_b = await run_coordinator(runner, "세계에서 가장 높은 산은 무엇인가요?")
    print(f"최종 출력 B: {result_b}")

    result_c = await run_coordinator(runner, "무작위 사실 하나 말해줘.")  # Info로 가야 함
    print(f"최종 출력 C: {result_c}")

    result_d = await run_coordinator(runner, "다음 달 도쿄행 항공편을 찾아줘.")  # Booker로 가야 함
    print(f"최종 출력 D: {result_d}")

In [8]:
if __name__ == "__main__":
    import nest_asyncio
    nest_asyncio.apply()
    await main()

--- Google ADK 라우팅 예제 (ADK 오토-플로우 스타일) ---
참고: Google ADK가 설치되어 있고 인증되어 있어야 합니다.

--- Running Coordinator with request: '파리에서 호텔을 예약해 주세요.' ---


-------------------------- 예약 핸들러 호출됨 ----------------------------
Coordinator Final Response: 파리에서 호텔 예약이 완료되었습니다.

최종 출력 A: 파리에서 호텔 예약이 완료되었습니다.


--- Running Coordinator with request: '세계에서 가장 높은 산은 무엇인가요?' ---


Unclosed connector
connections: ['deque([(<aiohttp.client_proto.ResponseHandler object at 0x172958890>, 966651.331821)])']
connector: <aiohttp.connector.TCPConnector object at 0x172982600>


-------------------------- 정보 핸들러 호출됨 ----------------------------
Coordinator Final Response: 세계에서 가장 높은 산은 에베레스트 산입니다.
최종 출력 B: 세계에서 가장 높은 산은 에베레스트 산입니다.

--- Running Coordinator with request: '무작위 사실 하나 말해줘.' ---


Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x1728d2e70>
Unclosed connector
connections: ['deque([(<aiohttp.client_proto.ResponseHandler object at 0x1667237d0>, 966649.257882875)])']
connector: <aiohttp.connector.TCPConnector object at 0x1728d3c80>
Task was destroyed but it is pending!
task: <Task pending name='Task-32' coro=<AsyncClient.aclose() running at /Users/kwangmyung/Desktop/project/agentic-design-patterns/.venv/lib/python3.12/site-packages/google/genai/client.py:90>>
/Users/kwangmyung/.local/share/uv/python/cpython-3.12.9-macos-aarch64-none/lib/python3.12/asyncio/base_events.py:716: RuntimeWarning: coroutine 'AsyncClient.aclose' was never awaited
  self._ready.clear()
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x172980830>
Unclosed connector
connections: ['deque([(<aiohttp.client_proto.ResponseHandler object at 0x172958290>, 966650.366446916)])']
connector: <aiohttp.connector.TCPConnector object at 0x1729

-------------------------- 정보 핸들러 호출됨 ----------------------------
Coordinator Final Response: 재미있는 사실 하나는 문어에게는 심장이 세 개 있다는 것입니다.

최종 출력 C: 재미있는 사실 하나는 문어에게는 심장이 세 개 있다는 것입니다.


--- Running Coordinator with request: '다음 달 도쿄행 항공편을 찾아줘.' ---


Task exception was never retrieved
future: <Task finished name='Task-62' coro=<AsyncClient.aclose() done, defined at /Users/kwangmyung/Desktop/project/agentic-design-patterns/.venv/lib/python3.12/site-packages/google/genai/client.py:90> exception=RuntimeError("Task <Task pending name='Task-62' coro=<AsyncClient.aclose() running at /Users/kwangmyung/Desktop/project/agentic-design-patterns/.venv/lib/python3.12/site-packages/google/genai/client.py:115>> got Future <Task pending name='Task-63' coro=<_wait_for_close() running at /Users/kwangmyung/Desktop/project/agentic-design-patterns/.venv/lib/python3.12/site-packages/aiohttp/connector.py:136>> attached to a different loop")>
Traceback (most recent call last):
  File "/Users/kwangmyung/.local/share/uv/python/cpython-3.12.9-macos-aarch64-none/lib/python3.12/asyncio/tasks.py", line 316, in __step_run_and_handle_result
    result = coro.throw(exc)
             ^^^^^^^^^^^^^^^
  File "/Users/kwangmyung/Desktop/project/agentic-design-patterns/

-------------------------- 예약 핸들러 호출됨 ----------------------------
Coordinator Final Response: 다음 달 도쿄행 항공편을 검색 중입니다.
최종 출력 D: 다음 달 도쿄행 항공편을 검색 중입니다.


이 스크립트는 메인 Coordinator 에이전트와 두 개의 전문 하위 에이전트인 Booker와 Info로 구성됩니다.   
각 전문 에이전트는 행동을 시뮬레이션하는 Python 함수를 래핑하는 FunctionTool을 갖추고 있습니다.  
- booking_handler 함수는 항공편 및 호텔 예약 처리를 시뮬레이션하고, info_handler 함수는 일반 정보 검색을 시뮬레이션합니다.  
- unclear_handler는 코디네이터가 위임할 수 없는 요청에 대한 폴백으로 포함되어 있지만, 현재 코디네이터 로직은 메인 run_coordinator 함수에서 위임 실패에 대해 명시적으로 사용하지 않습니다.


instruction에 정의된 Coordinator 에이전트의 주요 역할은 들어오는 사용자 메시지를 분석하고 Booker 또는 Info 에이전트에 위임하는 것입니다.  
이 위임은 Coordinator가 sub_agents를 정의했기 때문에 ADK의 Auto-Flow 메커니즘에 의해 자동으로 처리됩니다.  
run_coordinator 함수는 InMemoryRunner를 설정하고, 사용자 및 세션 ID를 생성한 다음, runner를 사용하여 코디네이터 에이전트를 통해 사용자의 요청을 처리합니다. [runner.run](http://runner.run) 메서드는 요청을 처리하고 이벤트를 생성하며, 코드는 event.content에서 최종 응답 텍스트를 추출합니다.

main 함수는 다양한 요청으로 코디네이터를 실행하여 시스템의 사용을 보여주며, 예약 요청을 Booker에, 정보 요청을 Info 에이전트에 어떻게 위임하는지를 보여줍니다.